In [6]:
import talib as ta
from datetime import datetime
from datetime import timedelta
from bigtrader.constant import Direction, OrderType

STRATEGY_NAME = "STRATEGY_1m"
        
def initialize(context):
    """初始化"""
    print("initialize")  
#     context.symbol = instruments
    context.my_instruments = context.get_conf_param("instruments")#从传入参数中获取需要交易的合约
    context.closetime_day = context.get_conf_param("closetime_day")#日内策略白盘平仓时间，一般14:58
    context.closetime_night = context.get_conf_param("closetime_night")#日内策略夜盘平仓时间，一般22:58，注意有些商品夜盘收盘时间不一样
    context.order_num = context.get_conf_param("order_num")#下单手数
    context.set_universe(context.my_instruments)#设置需要处理的合约

def before_trading(context, data):
    """盘前处理"""
    context.subscribe(context.my_instruments) #注册合约
    context.index = 0
    context.flag = 1 #用于获取今开
    context.count = 0
    # 前一交易日的最高和最低价为history_data里面的倒数第二条中取到
    context.history_data_for_4target = data.history(context.my_instruments, ["high","low"], 1, "1d")
    context.high = context.history_data_for_4target.iloc[0]['high']
    context.low = context.history_data_for_4target.iloc[0]['low']
#     for instr in context.my_instruments:
#         index = instr.find('.')
#         #获取当前主力合约，如果含有.即指定某个具体合约，否则认为是需要处理主连合约
#         if index<=0:
#             context.current_instruments.append(data.current_dominant(instr))
#         else:#使用指定合约
#             context.current_instruments.append(instr)                                   
#     print("当前主力合约", context.current_instruments)
    
def handle_data(context, data):
    """Bar行情推送"""
    #print("handle_data:", data.current_dt)
    #获取当前时间
    if context.flag == 1 :
        context.history_data = data.history(context.my_instruments, ["open"], 1, "1m")
        context.flag = context.flag + 1
        context.today_open = context.history_data.iloc[-1]['open'] 
    cur_date =  data.current_dt
    cur_hm = cur_date.strftime('%H:%M')
    #部分品种夜盘收盘时间不一样，此时间表示指定的尾盘平仓时间往后偏移30分钟，这段时间内不能开新仓，只能平仓。给30分钟是为了足够的冗余
    closetime_nightshift = (datetime.strptime(context.closetime_night,'%H:%M') + timedelta(minutes = 30)).strftime('%H:%M')
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(context.my_instruments, Direction.LONG)
    position_short = context.get_position(context.my_instruments, Direction.SHORT)
    # 获取当前价格
    price = data.current(context.my_instruments, "close")
    #尾盘平仓
    if((cur_hm>=context.closetime_day and cur_hm<="15:00") or (cur_hm>=context.closetime_night and cur_hm<=closetime_nightshift)):
        if(position_long.current_qty != 0):
            rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        if(position_short.current_qty != 0):
            rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) #输出关键日志
        #尾盘不开新仓，直接返回
        return
    # 多头持仓小于开盘价止损 平多
    c1 = position_long.current_qty != 0 and price < context.today_open
    # 空头持仓大于开盘价止损 平空
    c2 = position_short.current_qty != 0 and price > context.today_open
    # 当前的最新价大于了前一天的最高价且没有持仓 开多
    c3 = (position_short.current_qty == 0 and position_long.current_qty == 0) and (price > context.high and not context.count)
    # 当前最新价小于了前一天的最低价且没有持仓 开空
    c4 = (position_short.current_qty == 0 and position_long.current_qty == 0) and (price < context.low and not context.count)
    if c1:
        rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
        msg = str(data.current_dt) +  " 平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
        context.write_log(msg, stdout=1) 
        print("price",price,"context.high",context.high,"context.today_open",context.today_open)
    if c2:
        rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
        msg = str(data.current_dt) +  " 平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
        context.write_log(msg, stdout=1) 
    if c3:
        rv = context.buy_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
        msg = str(data.current_dt) +  " 开多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
        context.count = 1 #只开仓一次的判断
        context.write_log(msg, stdout=1) 
        print("price",price,"context.high",context.high,"context.today_open",context.today_open)

    if c4:
        rv = context.sell_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
        msg = str(data.current_dt) +  " 开空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
        context.count = 1
        context.write_log(msg, stdout=1) 

            
def handle_order(context, order):
    """委托回报推送"""
    msg = "handle_order:" + order.log_str()
    context.write_log(msg, stdout=1) 
         
def handle_trade(context, trade):
    """成交回报推送"""
    msg = "handle_trade:" + trade.log_str()
    context.write_log(msg, stdout=1) 
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(trade.symbol, Direction.LONG)
    position_short = context.get_position(trade.symbol, Direction.SHORT)
    msg = "当前多头持仓:"+ str(position_long) + "当前空头持仓:"+ str(position_short)
    context.write_log(msg, stdout=1) 

instruments = "RB2110.SHF" #["RB2110.SHF"] #["RB2110.SHF"] [I2105.DCE"] ['RB']# RB2101.SHF 如果想用主连合约，可以出入合约简码，例如RB,I,CF

#需要交易者传入的参数
strategy_setting = [
    {
        "instruments": instruments,
        "order_num": 2,
        "closetime_day": "14:58",
        "closetime_night": "22:58"
    }    
    
]
start_date = "2021-04-06"
end_date = "2021-05-12"
md = M.hfbacktest.v1(start_date=start_date,
                     end_date=end_date,
                     instruments=[instruments], #只传入一个合约便于策略逻辑展示
                     capital_base=100000,
                     product_type=Product.FUTURE,
                     frequency=Frequency.MINUTE,
                     initialize=initialize,
                     before_trading_start=before_trading,
                     handle_data=handle_data,
                     handle_order=handle_order,
                     handle_trade=handle_trade,
                     plot_charts=True,
                     volume_limit=1.0,
                     disable_cache=0,
                     show_debug_info=1,
                     strategy_setting=strategy_setting,
                     slippage_type=SlippageType.FIXED,#滑点固定模式
                     slippage_value=1.0,#买卖双向各1个滑点
                     m_deps=np.random.rand())

[2021-05-21 14:31:28.235736] INFO: moduleinvoker: hfbacktest.v1 开始运行..

[2021-05-21 14:31:28.243823] INFO: hfbacktest: passed-in daily_data_ds:None

[2021-05-21 14:31:28.246302] INFO: hfbacktest: passed-in minute_data_ds:None

[2021-05-21 14:31:28.248971] INFO: hfbacktest: passed-in tick_data_ds:None

[2021-05-21 14:31:28.251091] INFO: hfbacktest: passed-in each_data_ds:None

...

2021-05-21 14:31:37.002692 strategy_20210521(bkt999,): handle_order:[bkt999,39,RB2110.SHF,long,open,0,2,5708.0,pending,21:04:00,39] 
2021-05-21 14:31:37.003000 strategy_20210521(bkt999,): 2021-05-07 21:04:00 开多 for RB2110.SHF 最新价=5708.0 rv:0 
price 5708.0 context.high 5698.0 context.today_open 5677.0
2021-05-21 14:31:37.005138 strategy_20210521(bkt999,): handle_order:[bkt999,39,RB2110.SHF,long,open,2,2,5708.0,filled,21:04:00,39] 
2021-05-21 14:31:37.006420 strategy_20210521(bkt999,): handle_trade:[bkt999,39,RB2110.SHF,long,open,2,5708.0,SHFE.39,21:05:00.000000] 
2021-05-21 14:31:37.006651 strategy_20210521(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:2,avail_qty:2,cost_price:5708.0,last_price:5708.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-21 14:31:37.016587 strategy_20210521(bkt999,): handle_order:[bkt999,40,RB2110.SHF,short,close_today,0,2,5674.0,pending,21:25:00,40] 
2021-05-21 14:31:37.016837 strategy_

2021-05-21 14:31:37.232047 strategy_20210521(bkt999,): handle_order:[bkt999,41,RB2110.SHF,long,open,0,2,6195.0,pending,21:01:00,41] 
2021-05-21 14:31:37.232860 strategy_20210521(bkt999,): 2021-05-10 21:01:00 开多 for RB2110.SHF 最新价=6195.0 rv:0 
price 6195.0 context.high 6012.0 context.today_open 6113.0
2021-05-21 14:31:37.236425 strategy_20210521(bkt999,): handle_order:[bkt999,41,RB2110.SHF,long,open,2,2,6195.0,filled,21:01:00,41] 
2021-05-21 14:31:37.238784 strategy_20210521(bkt999,): handle_trade:[bkt999,41,RB2110.SHF,long,open,2,6194.0,SHFE.41,21:02:00.000000] 
2021-05-21 14:31:37.239082 strategy_20210521(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:2,avail_qty:2,cost_price:6194.0,last_price:6194.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-21 14:31:37.243929 strategy_20210521(bkt999,): handle_order:[bkt999,42,RB2110.SHF,short,close_today,0,2,6101.0,pending,21:05:00,42] 
2021-05-21 14:31:37.245252 strategy_

2021-05-21 14:31:37.689477 run_backtest() finished! time cost 5.8s! 


[2021-05-21 14:31:38.776482] INFO: hfbacktest: backtest done, raw_perf_ds:DataSource(ae8109fd20d746079a35acc48a88aa1bT)

[2021-05-21 14:31:39.285630] INFO: moduleinvoker: hfbacktest.v1 运行完成[11.04993s].